# Aritmética de punto flotante

Ejecutar este documento en forma dinámica: [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/manuxch/intro2prog/master?filepath=variables_tipos/variablesTipos.ipynb)

Vamos a hacer una cuenta simple:

In [1]:
0.1 + 0.1 + 0.1 - 0.3

5.551115123125783e-17

En la matemática que aprendimos en la escuela, $a + a + a - 3 a = 0$, pero cuando ejecutamos la instrucción de arriba el valor obtenido **no es cero** (aunque es muy pequeño).

Esto no es un error de Python, sino que ocurre debido a la representación interna que hacen las computadoras de los números que hace que no haya forma de representar **exactamente** valores como `0.1`. Esta representación interna es de [punto flotante](https://es.wikipedia.org/wiki/Coma_flotante) [binario](https://es.wikipedia.org/wiki/Sistema_binario). Vamos a ver primero estos conceptos, y luego los problemas que ocurren debido a esta representación finita de los números reales.

## Punto flotante decimal

Dado que la memoria de las computadoras es limitada, no es posible almacenar números con precisión infinita. Por lo tanto, de algún modo es necesario realizar un corte en la secuencia de dígitos que representa un números real. Esto es algo que hacemos rutinariamente en todos nuestros cálculos. Al número $1/2$ lo podemos representar **exactamente** como `0.5`, pero al número $1/3$ podemos representarlo, con **precisión** creciente, como:

|     |
| :--- |
|`0.3`|
|`0.33`|
|`0.333`|
|`0.3333`|
|`0.33333`|
|`...`|

La representación de punto flotante es una forma de notación científica usada en las computadoras con la cual se pueden representar números reales extremadamente grandes y pequeños de una manera muy eficiente y compacta, y con la que se pueden realizar operaciones aritméticas. El estándar actual para la representación en coma flotante es el [IEEE 754](https://es.wikipedia.org/wiki/IEEE_coma_flotante).

Esta representación consiste en descomponer un número en tres partes, de la siguiente forma:

$$r = (-1)^s c \; b^e$$

donde $r$ es el número que queremos representar, $s$ es el signo, $c$ es la mantisa, $b$ es la base de representación (10: decimal, 2: binario) y $e$ es el exponente. Por ejemplo:

\begin{align}
s &= 0 \\
c &= 31415926 \\
b &= 10 \\
e &= -7 \\
r &= (-1)^0 31415926 \; 10^{(-7)} = 3.1415926
\end{align}

Este formato satisface los siguientes requisitos:

- Puede representar números de órdenes de magnitud enormemente dispares (limitado por la longitud del exponente).
- Proporciona la misma precisión relativa para todos los órdenes (limitado por la longitud de la mantisa).
- Permite cálculos entre magnitudes: multiplicar un número muy grande y uno muy pequeño conserva la precisión de ambos en el resultado.

Los números de punto flotante decimales se expresan normalmente en notación científica con un punto explícito siempre entre el primer y el segundo dígitos. El exponente o bien se escribe explícitamente incluyendo la base, o se usa una e (o E) para separarlo de la mantisa.

# Punto flotante binario

Las computadoras utilizan el mismo formato pero binario, es decir, $b = 2$. Los formatos más utilizados son de 32 o 64 bits en total (4 u 8 bytes):

|Precisión | Bits | Bits significativos | Bits del exponente | Número más pequeño | Número más grande |
| --- | --- | --- | --- | --- | --- |
|Simple | 32 | 23 + 1 signo | 8 | `~1.2e-38` | `~3.4e38` |
|Doble | 64 | 52 + 1 signo | 11 | `~5.0e-324` | `~1.8e308` |

Algunas características:

- En la secuencia de bits, el primero es el del signo, luego el exponente y finalmente la mantisa.
- El exponente no tiene signo, en su lugar se le resta un desplazamiento (*bias*) (127 para simple y 1023 para doble precisión). Esto, junto con la secuencia de bits, permite que los números de punto flotante se puedan comparar y ordenar correctamente incluso cuando se interpretan como enteros.
- Se asume que el bit más significativo de la mantisa es 1 y se omite, excepto para casos especiales.
- Hay valores diferentes para cero positivo y cero negativo. Estos difieren en el bit del signo, mientras que todos los demás son 0. Deben ser considerados iguales aunque sus secuencias de bits sean diferentes.
- Hay valores especiales no numéricos (NaN, *not a number* en inglés) en los que los bits del exponente son todos `1` y los de la mantisa no son todos `0`. Estos valores representan el resultado de algunas operaciones indefinidas (como multiplicar 0 por infinito, operaciones que involucren NaN, o casos específicos). Incluso valores NaN con idéntica secuencia de bits no deben ser considerados iguales.


# Repaso (?): conversión de decimal a binario

Para convertir un número decimal `n`, por ejemplo `2.71728` a binario con una cierta cantidad `k` de dígitos (por ejemplo 4), se deben seguir los pasos a continuación:

1. Convertir la parte entera de `n` en el binario equivalente:
    1. Dividir el número decimal por 2 y guadar el resto en una lista.
    1. Dividir el cociente por 2.
    1. Repetir el paso B hasta que el cociente sea 0.
    1. El número binario equivalente resulta de la lista del paso A en orden inverso.
    
Divisor | Cociente | Resto
--- | --- | ---
2 | 1 | 0
1 | 0 | 1

$$2_{10} = 10_{2}$$

2. Convertir la parte fraccionaria de `n` en el binario equivalente:
    1. Multiplicar la parte fraccionaria decimal por 2.
    1. La parte entera del resultado de A es el primer dígito de la parte fraccionaria binaria.
    1. Repetir el paso 1 usando solo la parte fraccional del número y luego el paso B, hasta obtener los `k` dígitos.
    
| Fracción decimal | x 2 | Parte entera del resultado |
| --- | --- | --- |
| 0.71728 | 1.43456 | 1 |
| 0.43456 | 0.86912 | 0 |
| 0.86912 | 1.73824 | 1 |
| 0.73824 | 1.47648 | 1 |

$$0.71728_{10} = 1011_{2} $$

3. Concatenar la parte entera binaria con la fraccionaria binaria:

$$2.71728_{10} = 10.1011_{2}$$

Podemos hacer un script que realice el cálculo por nosotros:

In [5]:
n = 2.71728
k = 4
binario = ''
entero = int(n)
fraccional = n - entero
# Convertimos la parte entera a su formato binario
while(entero):
    resto = entero % 2
    binario += str(resto)
    entero //= 2
binario = binario[::-1]
binario += '.'
# Convertimos la parte fraccional a su formato binario
while(k):
    fraccional *= 2
    fraccional_bit = int(fraccional)
    if fraccional_bit == 1:
        fraccional -= fraccional_bit
        binario += '1'
    else:
        binario += '0'
    k -= 1
print(binario)

10.1011


## Errores

### Error de redondeo

### Error de comparación

### Error de propagación

---

Este notebook fue preparado a partir de las siguientes fuentes:

- [Lo que todo programador debería saber sobre aritmética de punto flotante](http://puntoflotante.org/).
- [What Every Computer Scientist Should Know About Floating-Point Arithmetic](https://docs.oracle.com/cd/E19957-01/806-3568/ncg_goldberg.html).
- [The Python tutorial: 15 - Floating Point Arithmetic: Issues and Limitations](https://docs.python.org/3/tutorial/floatingpoint.html).